In [1]:
import sys
sys.path.append('/mnt/nas/minici/UrbRecLoop')


In [2]:
%cd /mnt/nas/minici/UrbRecLoop

/mnt/nas/minici/UrbRecLoop


/home/minici/anaconda3/envs/frasson/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from model.recommender3 import MultiVAE, MatrixFactorization
from model import user
from datetime import timedelta
import glob
import tqdm
import os
import pandas as pd
import numpy as np

def filtering_training_data(training_dataset, min_activity):
    # Count the number of rows per user
    user_counts = training_dataset['uid'].value_counts()
    # Get the users with at least x rows
    valid_users = user_counts[user_counts >= min_activity].index
    # Filter the DataFrame to include only valid users
    filtered_dataset = training_dataset[training_dataset['uid'].isin(valid_users)]
    return filtered_dataset

def create_interaction_matrix(data):
    interaction_counts = data.groupby(['uid', 'venueID']).size().reset_index(name='counts')
    interaction_matrix = interaction_counts.pivot(index='uid', columns='venueID', values='counts').fillna(0)
    return interaction_matrix


def calculate_metrics(recommended_venues, real_visited_venues):
    recommended_venues = list(recommended_venues)
    real_visited_venues = set(real_visited_venues)

    # Calculate hits
    hits = [1 if venue in real_visited_venues else 0 for venue in recommended_venues]

    # Precision
    precision = sum(hits) / len(recommended_venues) if recommended_venues else 0

    # Recall
    recall = sum(hits) / len(real_visited_venues) if real_visited_venues else 0

    # NDCG
    dcg = 0
    idcg = 0
    for i, hit in enumerate(hits, 1):
        if hit:
            dcg += 1 / np.log2(i + 1)
    for i in range(1, len(real_visited_venues) + 1):
        idcg += 1 / np.log2(i + 1)
    ndcg = dcg / idcg if idcg > 0 else 0

    # MRR
    mrr = 0
    for i, hit in enumerate(hits, 1):
        if hit:
            mrr = 1 / i
            break

    # Hit Rate
    hit_rate = 1 if any(hits) else 0

    return {
        'ndcg': ndcg,
        'precision': precision,
        'recall': recall,
        'mrr': mrr,
        'hitrate': hit_rate
    }


In [4]:
# DATASET HYPER-PARAMETERS #
city = "nyc"
city_files = glob.glob('*_visits.csv')
data_file = f"./data/processed/foursquare_complete_{city}_cp_small.csv"
train_window = 60  # Training window in days
k_days = 7  # Epoch length in days
threshold = 100  # Performance degradation threshold (adjust as needed)
max_simulation_days = 90  # Maximum number of simulation days
min_activity = 5  
topK = 20

# RECSYS HYPER-PARAMETERS #
# MF
num_latent_factors = 32
# UserKNN
num_nearest_neighbors = 5
